# Filtering DSL & Non-DSL related Tickets

The main goal of this notebook is to filter DSL related issue tickets from tickets database for time span of 1st Jan 2020 to April 2021.



>Output data format expected:
* We would be expecting a dataframe with 3 unique columns i.e., **|assetid|incident_date|label|** which would provide us the DSL tickets reached to the maximum level(expensive solution) for any assetid.



>Steps involved:
* Get tickets data 
* Get DSL data for same time span and filter only those tickets which have dsl data in database
* Filter all cpe relacement and refurbushed cpe with Issues and keep noproblem one


## Table Of Contents:
* [1. Import packages and config](#sec1)
    * [1.1 Spark configuration](#sec2)
    * [1.2 Import Packages](#sec3)
    * [1.3 Setup style](#sec4)
* [2. Load tickets data](#sec5)
* [3. Load pol agg data](#sec6)
* [4. Load different data](#sec7)
    * [4.1 Load Replacement data](#sec8)
    * [4.2 Load Refurbishment data](#sec9)
    * [4.3 Join Refurbishment & Replacement data](#sec10)
    * [4.4 Filter CPEs with "No Problem"](#sec11)
    * [4.5 Filter CPES with all issues from the data](#sec12)
* [5. Filter DSL ticketds data](#sec13)
    * [5.1 Filter DSL tickets data with all levels](#sec14)
* [6. Filter Non-DSL tickets data](#sec15)

## 1. Import packages and config <a class="anchor" id="sec1"></a>

###  1.1 Spark configuration <a class="anchor" id="sec2"></a>
Lets configure our spark session

In [1]:
%%configure -f
{"conf":
 {"spark.driver.cores": "6",
  "spark.driver.memory": "14g",
  "spark.executor.cores": "6",
  "spark.executor.memory": "14g",
  "spark.dynamicAllocation.enabled": "true",
  "spark.dynamicAllocation.minExecutors" : "4",
  "spark.driver.maxResultSize": "4g"
    }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1572,application_1620749185953_146498,pyspark,idle,Link,Link,


###  1.2 Import packages <a class="anchor" id="sec3"></a>

In [2]:
# Data Science Packages
import sys
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from  matplotlib import pyplot
import seaborn as sns
import warnings
from datetime import datetime
from datetime import timedelta
import datetime


#Spark Packages
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import DataFrame
from pyspark.sql import types 
import pyspark.sql.types
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.functions import col

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1573,application_1620749185953_146533,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

###  1.3  Let's setup some style!<a class="anchor" id="sec3"></a>
This is not a requirement but anyway you know that they say:
> "Fashions fade, style is eternal." <br/>
_Yves Saint Laurent_

In [3]:
# Matplotlib

#matplotlib.use('agg')
plt.switch_backend('agg')

# Seaborn Style
sns.set(style='ticks')
sns.set_style({'font.family': 'Hiragino Maru Gothic Pro'})
sns.set_palette("cool")

# Pandas Style
pd.set_option("display.max_column", 9999)
pd.set_option("display.max_row", 9999)

# Ignore annoying warning 
warnings.filterwarnings('ignore')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

##  2. Loading Tickets Data  <a class="anchor" id="sec5"></a>

Let's load tickets data with some necessary fields

In [4]:
path_tickets_1 = "hdfs://nameservicedev1//user/dt_srajan/predictive_care/tickets/Smetnje_0107-1510_2020.csv"
df_tickets_1 = spark.read.option("encoding", "ISO-8859-1").option("inferSchema", "true").csv(path_tickets_1,header = True,sep=";")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df_tickets_1.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['SOURCE', 'SOURCE_ID', 'TICKET_ID', 'START_PERIOD', 'START_DATE', 'START_DATE_TIME_DONAT', 'FAULT_END_DATE', 'FAULT_END_DATE_TIME', 'COMPLAINT_PROCESS_END_DATE', 'COMPLAINT_PROCESS_END_TIME', 'DISPETCH_DATE', 'DISPETCH_TIME', 'DISPETCH_DATE_TIME', 'ASSET_ID', 'EXEC_REG', 'EXEC_GA', 'EXEC_FG', 'EXEC_FG_DESCR', 'EXECUTOR_ID', 'EXECUTOR_NAME', 'S_REG', 'FRC_SERVICE', 'SERVICE_TYPE', 'SEGMENT_ID', 'COMPLAINT_STATUS_ID', 'COMPL_KEY_STATION_ID', 'COMPL_KEY_REASON_ID', 'COMPL_KEY_LOGOFF_ID', 'COMPL_KEY_TYPE_ID', 'COMPL_KEY_DEFLECTION_ID', 'FAULT_DURATION', 'KPI_DURATION', 'FAULT_HANDLING_DURATION', 'NETWORK_DURATION', 'CC_DURATION', 'FAULT_END_TO_END_DURATION', 'TECHNOLOGY', 'CLEARED_TEMPORARELY', 'SMETNJA_GRUPIRANA', 'SMETNJA_OTKLONJENA_IZLASKOM', 'RISING_SITE', 'COMPL_DISCONN', 'NUMBER_COME_BACK', 'FAULT_RESOLUTION', 'WORK_CODE', 'ACTION_TYPE', 'COMPLAINT_SOURCE', 'ADSL_MODEM_CHANGE', 'ADSL_MODEM_PRICE', 'DISPETCH_SMC', 'FR_DISPETCH_EXEC', 'FR_DISPETCH_SMC', 'SUMMARY', 'FDM', 'CX_TYPE', 'I

In [7]:
df_tickets_1p = df_tickets_1.toPandas()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
df_tickets_1p['ACTION_TYPE'].unique()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

array([u'Uklanjanje smetnje ADSL (Max TV) - prekid',
       u'Uklanjanje smetnje ADSL (Max TV) - prekid - SMART',
       u'Uklanjanje smetnje ULL - prekid- Komercijalna, Regulirana',
       u'Uklanjanje smetnje ADSL (VoIP, Max TV) - prekid- Hybrid Access',
       u'Uklanjanje smetnji (FTTH:Internet,Net Phone paketi) - SMART',
       u'Uklanjanje smetnje ADSL (VoIP, Max TV) - prekid, nije migriran IPTV korisnik',
       u'Zamjena opreme preko tehni\xe8ara- Hybrid Access',
       u'Uklanjanje smetnje ADSL (VoIP, Max TV) - degradacija- Premium podr\x9aka',
       u'Uklanjanje smetnje ADSL (VoIP) - prekid',
       u'Uklanjanje smetnje Bitstream - prekid- Regulirana, Komercijalna',
       u'Uklanjanje smetnje ADSL (VoIP, Max TV) - prekid',
       u'Uklanjanje smetnji (FTTH:Internet,Halo,MAXTV)',
       u'Uklanjanje smetnje (NetPhone paket) - prekid - SMART',
       u'Uklanjanje smetnje ADSL (VoIP) - degradacija',
       u'Uklanjanje smetnji (FTTH:Veleprodajni FTTH)- Komercijalna',
       u'

In [8]:
df_tickets_1p['FAULT_RESOLUTION'].unique() 'FAULT_RESOLUTION','Nacin_otklona','konacna_otklona'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

array([u'1.razina kod korisnika', u'Nepoznato', u'1.razina na mre\x9ei',
       u'Razina na agregaciji', u'Drugi operator',
       u'2.razina na mre\x9ei', u'Otkazana smetnja',
       u'Automatska dijagnostika', u'Daljinski otklonjena smetnja', None,
       u'Otklonjena smetnja - prekid - 1',
       u'Otklonjena smetnja - degradacija 45',
       u'Smetnja nije otklonjena - 35',
       u'Po ispitivanju dobar (gre\x9aka nije prona\xf0ena) - 2',
       u'Otkazana prijava (korisnik odjavio smetnju ili krivo prijavljena smetnja) - 30',
       u'Usluga ne zadovoljava o\xe8ekivanja korisnika (korisnik upu\xe6en) - 31',
       u'Privremeno otklonjena smetnja - 3', u'Nema podataka'],
      dtype=object)

In [9]:
df_tickets_1p['Nacin_otklona'].unique()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

array([u'Zamjena - 8', u'Nepoznato', u'Popravak - 1',
       u'Onemogu\xe6en pristup prostoru u vlasni\x9atvu korisnika - 5',
       u'Vlastito odr\x9eavanje korisnika - 6', u'Reset - 4',
       u'Rekonfiguracija - 3', u'Promjena profila - 2',
       u'\xc8i\x9a\xe6enje opti\xe8kih priklju\xe8aka - 7',
       u'Drugi operator - 9', None], dtype=object)

In [10]:
df_tickets_1p['Konacna_odjava'].unique()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

array([u'Otklonjena smetnja - prekid - 1', u'Nepoznato',
       u'Otklonjena smetnja - degradacija 45',
       u'Smetnja nije otklonjena - 35',
       u'Po ispitivanju dobar (gre\x9aka nije prona\xf0ena) - 2',
       u'Otkazana prijava (korisnik odjavio smetnju ili krivo prijavljena smetnja) - 30',
       u'Automatskom dijagnostikom je utvr\xf0ena gre\x9aka na korisni\xe8koj opremi',
       u'Privremeno otklonjena smetnja - 3',
       u'Usluga ne zadovoljava o\xe8ekivanja korisnika (korisnik upu\xe6en) - 31',
       u'Isklju\xe8enja zbog duga ili na zahtjev korisnika - 32',
       u'Automatskom dijagnostikom nije utvr\xf0eno postojanje smetnje',
       None], dtype=object)

In [11]:
df_tickets_1p['Mjesto_smetnje'].unique()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

array([u'STB - 44', u'DSL modem - 30', u'Nepoznato', u'Parica - 110',
       u'Nepoznato - 125', u'Oprema u vlasni\x9atvu korisnika - 83',
       u'DSLAM - 32', u'MDF/ODF - 69', u'Sat. antenski sustav - 102',
       u'Ostalo - 113', u'Spojna pristupna mre\x9ea',
       u'Instalacija u vlasni\x9atvu korisnika - 84', u'IP telefon - 56',
       u'T-Box ure\xf0aj-135', u'DSLAM port - 68', u'HA HGW IAD',
       u'Voice gateway - 55',
       u'Smetnja nije u podru\xe8ju odgovornosti HT-a - 86',
       u'Podzemni kabel - 70', u'ONT - 63', u'Izvod - 92',
       u'HDF - HT strana - 87',
       u'Posredni Kabel (izme\xf0u HDF - MDF) - 88',
       u'Uvodna kutija na objektu korisnika - 94',
       u'Zra\xe8ni kabel - 93', u'Platforma - 133', u'LTE modem',
       u'WiFi postavke na modemu - 164', u'DSLAM plo\xe8ica - 130',
       u'DSL spliter - 31', u'Ostalo - 160',
       u'Daljinski upravlja\xe8 - 58', u'FGSM oprema - 24',
       u'FTTB/FTTDP \xe8vor/izvod', u'SIM kartica za HA',
       u'DVB-T

In [4]:
path_tickets_1 = "hdfs://nameservicedev1//user/dt_srajan/predictive_care/tickets/Smetnje_0107-1510_2020.csv"
df_tickets_1 = spark.read.option("encoding", "ISO-8859-1").option("inferSchema", "true").csv(path_tickets_1,header = True,sep=";")
df_tickets_1 = df_tickets_1.select('TICKET_ID','START_DATE_TIME_DONAT','ASSET_ID','Uzrok_smetnje', 'WORK_CODE','Vrsta_prijavljene_smetnje','SOURCE','SOLUTION_LEVEL','SUMMARY','FAULT_RESOLUTION','Nacin_otklona','Konacna_odjava')

path_tickets_2 = "hdfs://nameservicedev1//user/dt_srajan/predictive_care/tickets/FAULTS_E2E_01102020-31122020.csv"
df_tickets_2 = spark.read.option("encoding", "ISO-8859-1").option("inferSchema", "true").csv(path_tickets_2,header = True,sep=";")
df_tickets_2 = df_tickets_2.select('TICKET_ID','START_DATE_TIME_DONAT','ASSET_ID','Uzrok_smetnje', 'WORK_CODE','Vrsta_prijavljene_smetnje','SOURCE','SOLUTION_LEVEL','SUMMARY','FAULT_RESOLUTION','Nacin_otklona','Konacna_odjava')

path_tickets_3 = "hdfs://nameservicedev1//user/dt_srajan/predictive_care/tickets/FAULTS_01012020-30062020.csv"
df_tickets_3 = spark.read.option("encoding", "ISO-8859-1").option("inferSchema", "true").csv(path_tickets_3,header = True,sep=";")
df_tickets_3 = df_tickets_3.select('TICKET_ID','START_DATE_TIME_DONAT','ASSET_ID','Uzrok_smetnje', 'WORK_CODE','Vrsta_prijavljene_smetnje','SOURCE','SOLUTION_LEVEL','SUMMARY','FAULT_RESOLUTION','Nacin_otklona','Konacna_odjava')

path_tickets_4 = "hdfs://nameservicedev1//user/dt_srajan/predictive_care/tickets/FAULTS_01012021-30042021.csv"
df_tickets_4 = spark.read.option("encoding", "ISO-8859-1").option("inferSchema", "true").csv(path_tickets_4,header = True,sep=";")
df_tickets_4 = df_tickets_4.select('TICKET_ID','START_DATE_TIME_DONAT','ASSET_ID','Uzrok_smetnje', 'WORK_CODE','Vrsta_prijavljene_smetnje','SOURCE','SOLUTION_LEVEL','SUMMARY','FAULT_RESOLUTION','Nacin_otklona','Konacna_odjava')

df_tickets_1 = df_tickets_1.select('TICKET_ID','START_DATE_TIME_DONAT','ASSET_ID','Uzrok_smetnje','WORK_CODE','Vrsta_prijavljene_smetnje','SOURCE','SOLUTION_LEVEL','SUMMARY','FAULT_RESOLUTION','Nacin_otklona','Konacna_odjava')
df_tickets_1 = df_tickets_1.withColumn('START_DATE_TICKET',F.unix_timestamp('START_DATE_TIME_DONAT', "yyyy-MM-dd'T'HH:mm:ss").cast(types.TimestampType()))
df_tickets_1 = df_tickets_1.withColumn('START_DATE_TICKET',F.date_format(F.col("START_DATE_TICKET"), "yyy-MM-dd HH:mm:ss"))

df_tickets_2 = df_tickets_2.select('TICKET_ID','START_DATE_TIME_DONAT','ASSET_ID','Uzrok_smetnje','WORK_CODE','Vrsta_prijavljene_smetnje','SOURCE','SOLUTION_LEVEL','SUMMARY','FAULT_RESOLUTION','Nacin_otklona','Konacna_odjava')
df_tickets_2 = df_tickets_2.withColumn('START_DATE_TICKET',F.unix_timestamp('START_DATE_TIME_DONAT', "yyyy-MM-dd HH:mm:ss").cast(types.TimestampType()))
df_tickets_2 = df_tickets_2.withColumn('START_DATE_TICKET',F.date_format(F.col("START_DATE_TICKET"), "yyy-MM-dd HH:mm:ss"))

df_tickets_3 = df_tickets_3.select('TICKET_ID','START_DATE_TIME_DONAT','ASSET_ID','Uzrok_smetnje','WORK_CODE','Vrsta_prijavljene_smetnje','SOURCE','SOLUTION_LEVEL','SUMMARY','FAULT_RESOLUTION','Nacin_otklona','Konacna_odjava')
df_tickets_3 = df_tickets_3.withColumn('START_DATE_TICKET',F.unix_timestamp('START_DATE_TIME_DONAT', "yyyy-MM-dd'T'HH:mm:ss").cast(types.TimestampType()))
df_tickets_3 = df_tickets_3.withColumn('START_DATE_TICKET',F.date_format(F.col("START_DATE_TICKET"), "yyy-MM-dd HH:mm:ss"))

df_tickets_4 = df_tickets_4.select('TICKET_ID','START_DATE_TIME_DONAT','ASSET_ID','Uzrok_smetnje','WORK_CODE','Vrsta_prijavljene_smetnje','SOURCE','SOLUTION_LEVEL','SUMMARY','FAULT_RESOLUTION','Nacin_otklona','Konacna_odjava')
df_tickets_4 = df_tickets_4.withColumn('START_DATE_TICKET',F.unix_timestamp('START_DATE_TIME_DONAT', "yyyy-MM-dd'T'HH:mm:ss").cast(types.TimestampType()))
df_tickets_4 = df_tickets_4.withColumn('START_DATE_TICKET',F.date_format(F.col("START_DATE_TICKET"), "yyy-MM-dd HH:mm:ss"))

df_tickets = df_tickets_1.union(df_tickets_2)
df_tickets = df_tickets.union(df_tickets_3)
df_tickets = df_tickets.union(df_tickets_4)
df_tickets = df_tickets.dropDuplicates()
df_tickets = df_tickets.withColumnRenamed('Uzrok_smetnje', 'TICKET_TENTATIVE_ROOT_CAUSE')
df_tickets = df_tickets.withColumnRenamed('Vrsta_prijavljene_smetnje', 'Type_of_reported_interference')
df_tickets = df_tickets.withColumnRenamed('Nacin_otklona', 'deviation_mode')
df_tickets = df_tickets.withColumnRenamed('Konacna_odjava', 'final_logout')
df_tickets = df_tickets.drop('START_DATE_TIME_DONAT')

df_tickets = df_tickets.dropDuplicates()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df_tickets.describe().select('START_DATE_TICKET').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+
|  START_DATE_TICKET|
+-------------------+
|            1218714|
|               null|
|               null|
|2020-01-01 00:13:34|
|2021-04-30 23:57:23|
+-------------------+

In [6]:
print('Number of Unique AssetID in tickets dattabase:',df_tickets.select('ASSET_ID').distinct().count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

('Number of Unique AssetID in tickets dattabase:', 658190)

##  3. Loading Poll day agg data<a class="anchor" id="sec6"></a>

Load pol aggregated dat for the same timeframe as tickets dataset

In [5]:
def rename_cols(df, prefix):
    """Column renameb by adding a prefix.
    
    Args:
        df (dataframe): Input dataframe.
        
    Returns:
        df (dataframe): Output datframe witha ppended prefix to columns.
    """
    for feature in df.columns:
        df = df.withColumnRenamed(feature,prefix+feature)
    return df

def parsing_date_format(datetime_str, target_date_format):
    """Fuction to modify the date format.
    
    Args:
        datetime_str (str): Date in original format. expected "%Y-%m-%d".
        target_date_format (str): Desire date format e.g "%Y%m%d".
    
    Returns
        str: Date in target format.
    """
    date_obj = datetime.strptime(datetime_str, "%Y-%m-%d").date()
    return date_obj.strftime(target_date_format)

def data_import(database, table, start_time, end_time, spark):
    """Function to load data from Hive tables based on start time and end time.
    
    Args:
        database (string): Database name in HIVE.
        table (string): Table name in HIVE.
        start_time (str): Start time for the query.
        end_time (str): End time for the query.
        spark (obj): Spark session object.
        
    Returns:
        dataframe :  spark dataframe with loaded data.
    """
    #start_time = parsing_date_format(start_time, "%Y-%m-%d")
    #end_time = parsing_date_format(end_time, "%Y-%m-%d")

    df = spark.sql("select * from {0}.{1} where {2} between '{3}' and '{4}' " \
              .format(database, table, 'datum', start_time, end_time))
    return df

def spark_init():
    """Init Spark session.
    Returns:
        object: spark session.
    """
    spark = SparkSession.builder \
        .master('yarn') \
        .appName('predictive_care') \
        .enableHiveSupport() \
        .getOrCreate()
    spark.sparkContext.setLogLevel("ERROR")
    return spark

spark=spark_init()
start_time, end_time = '2020-01-01', '2021-04-30'
df_dslam = data_import('cdl_blos', 'pol_day_aggregation', start_time, end_time, spark) #per day

    
#dropping missing keys
df_dslam =df_dslam.na.drop(subset=['assetid','datum'])
    
# Drop multiple samples per day
df_dslam = df_dslam.dropDuplicates(['assetid','datum'])
    
#Renamming for conditional join
df_dslam = rename_cols(df_dslam, 'dslam_')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
df_dslam.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['dslam_date_inserted', 'dslam_dns_device', 'dslam_slot', 'dslam_port', 'dslam_ip_device', 'dslam_dis_device', 'dslam_distinct_modulation', 'dslam_no_of_dominant_modulation', 'dslam_code_of_dominant_modulation', 'dslam_avg_bitrate_us', 'dslam_max_bitrate_us', 'dslam_min_bitrate_us', 'dslam_avg_bitrate_ds', 'dslam_max_bitrate_ds', 'dslam_min_bitrate_ds', 'dslam_avg_attenuation_ds', 'dslam_max_attenuation_ds', 'dslam_min_attenuation_ds', 'dslam_avg_attenuation_us', 'dslam_max_attenuation_us', 'dslam_min_attenuation_us', 'dslam_avg_power_us', 'dslam_max_power_us', 'dslam_min_power_us', 'dslam_avg_power_ds', 'dslam_max_power_ds', 'dslam_min_power_ds', 'dslam_avg_att_bitrate_us', 'dslam_max_att_bitrate_us', 'dslam_min_att_bitrate_us', 'dslam_avg_att_bitrate_ds', 'dslam_max_att_bitrate_ds', 'dslam_min_att_bitrate_ds', 'dslam_avg_snr_us', 'dslam_max_snr_us', 'dslam_min_snr_us', 'dslam_avg_snr_ds', 'dslam_max_snr_ds', 'dslam_min_snr_ds', 'dslam_no_counts', 'dslam_avg_bandline_ds', 'dslam_max_b

In [6]:
df_ticket_filtered_dsl = df_tickets.join(df_dslam,df_tickets.ASSET_ID == df_dslam.dslam_assetid, how = 'inner')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
print('Number of assetid which are related to DSL issues:',df_ticket_filtered_dsl.select('ASSET_ID').distinct().count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

('Number of assetid which are related to DSL issues:', 320253)

In [7]:
df_ticket_filtered_dsl = df_ticket_filtered_dsl.drop('dslam_date_inserted', 'dslam_dns_device', 'dslam_slot', 'dslam_port', 'dslam_ip_device', 'dslam_dis_device', 'dslam_distinct_modulation', 'dslam_no_of_dominant_modulation', 'dslam_code_of_dominant_modulation', 'dslam_avg_bitrate_us', 'dslam_max_bitrate_us', 'dslam_min_bitrate_us', 'dslam_avg_bitrate_ds', 'dslam_max_bitrate_ds', 'dslam_min_bitrate_ds', 'dslam_avg_attenuation_ds', 'dslam_max_attenuation_ds', 'dslam_min_attenuation_ds', 'dslam_avg_attenuation_us', 'dslam_max_attenuation_us', 'dslam_min_attenuation_us', 'dslam_avg_power_us', 'dslam_max_power_us', 'dslam_min_power_us', 'dslam_avg_power_ds', 'dslam_max_power_ds', 'dslam_min_power_ds', 'dslam_avg_att_bitrate_us', 'dslam_max_att_bitrate_us', 'dslam_min_att_bitrate_us', 'dslam_avg_att_bitrate_ds', 'dslam_max_att_bitrate_ds', 'dslam_min_att_bitrate_ds', 'dslam_avg_snr_us', 'dslam_max_snr_us', 'dslam_min_snr_us', 'dslam_avg_snr_ds', 'dslam_max_snr_ds', 'dslam_min_snr_ds', 'dslam_no_counts', 'dslam_avg_bandline_ds', 'dslam_max_bandline_ds', 'dslam_min_bandline_ds', 'dslam_avg_bandline_us', 'dslam_max_bandline_us', 'dslam_min_bandline_us', 'dslam_avg_net_datarate_ds', 'dslam_max_net_datarate_ds', 'dslam_min_net_datarate_ds', 'dslam_avg_net_datarate_us', 'dslam_max_net_datarate_us', 'dslam_min_net_datarate_us', 'dslam_sum_cv_us', 'dslam_count_cv_us', 'dslam_sum_cv_ds', 'dslam_count_cv_ds', 'dslam_sum_es_ds', 'dslam_count_es_ds', 'dslam_sum_es_us', 'dslam_count_es_us', 'dslam_sum_ses_ds', 'dslam_count_ses_ds', 'dslam_sum_ses_us', 'dslam_count_ses_us', 'dslam_sum_fec_ds', 'dslam_count_fec_ds', 'dslam_sum_fec_us', 'dslam_count_fec_us', 'dslam_inits', 'dslam_platforma', 'dslam_model', 'dslam_dis_dslam_name', 'dslam_dis_dslam_slot_port', 'dslam_em_dslam_name', 'dslam_em_dslam_slot_port', 'dslam_card_type', 'dslam_port_access_id', 'dslam_status_porta', 'dslam_bandwidth', 'dslam_vdsl_adsl', 'dslam_vendor', 'dslam_assetid', 'dslam_status', 'dslam_port_inst_id', 'dslam_internet', 'dslam_iptv', 'dslam_voip', 'dslam_servis', 'dslam_regija', 'dslam_razdjelnik', 'dslam_rg_port', 'dslam_rg_port_inst_id', 'dslam_prim_izvod', 'dslam_prim_izvod_inst_id', 'dslam_prim_parica', 'dslam_sek_izvod', 'dslam_sek_parica', 'dslam_em_profil', 'dslam_datum')
df_ticket_filtered_dsl.limit(3).show(200,truncate=False, vertical=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-RECORD 0------------------------------------------------------------------------
 TICKET_ID                     | 46588210                                        
 ASSET_ID                      | 2095669                                         
 TICKET_TENTATIVE_ROOT_CAUSE   | Oteæenje zbog radova (Nisu radovi T-HT-a) - 35 
 WORK_CODE                     | FAULTREPAIR                                     
 Type_of_reported_interference | prekid                                          
 SOURCE                        | WWMS                                            
 SOLUTION_LEVEL                | 2                                               
 SUMMARY                       | DSL SINHRONIZACIJA - PREKID                     
 FAULT_RESOLUTION              | Daljinski otklonjena smetnja                    
 deviation_mode                | Reset - 4                                       
 final_logout                  | Otklonjena smetnja - prekid - 1                 
 START_DATE_TICK

In [13]:
df_ticket_filtered_dsl.groupBy("SOLUTION_LEVEL") \
    .agg(F.countDistinct("ASSET_ID").alias("count_assetid")).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+-------------+
|SOLUTION_LEVEL|count_assetid|
+--------------+-------------+
|          null|           14|
|             1|       130131|
|             3|       219888|
|             4|        34736|
|             2|        79467|
|             0|         2277|
+--------------+-------------+

## 4. Loading different data<a class="anchor" id="sec7"></a>

### 4.1 Loading Replacement data<a class="anchor" id="sec8"></a>

In [8]:
path_cpe_replacement1 = "hdfs://nameservicedev1//user/dt_srajan/predictive_care/replacement/cpe_replacement.csv"
df_cpe_replacement1 = spark.read.option("encoding", "ISO-8859-1").option("inferSchema", "true").csv(path_cpe_replacement1,header = True,sep=";")

path_cpe_replacement2 = "hdfs://nameservicedev1//user/dt_srajan/predictive_care/replacement/replacement_new_data/CPE replacement_2020_01-06.csv"
df_cpe_replacement2 = spark.read.option("encoding", "ISO-8859-1").option("inferSchema", "true").csv(path_cpe_replacement2,header = True,sep=";")

path_cpe_replacement3 = "hdfs://nameservicedev1//user/dt_srajan/predictive_care/replacement/replacement_new_data/CPE replacement_2021_01-07.csv"
df_cpe_replacement3 = spark.read.option("encoding", "ISO-8859-1").option("inferSchema", "true").csv(path_cpe_replacement3,header = True,sep=";")
df_cpe_replacement3 = df_cpe_replacement3.filter((df_cpe_replacement3.Month == 1) | (df_cpe_replacement3.Month == 2) | (df_cpe_replacement3.Month == 3) | (df_cpe_replacement3.Month == 4))

df_cpe_replacement1 = df_cpe_replacement1.select('TICKET_ID','ASSET_ID', 'CPE_SERIAL_NUMBER')
df_cpe_replacement2 = df_cpe_replacement2.select('TICKET_ID','ASSET_ID', 'CPE_SERIAL_NUMBER')
df_cpe_replacement3 = df_cpe_replacement3.select('TICKET_ID','ASSET_ID', 'CPE_SERIAL_NUMBER')
df_cpe_replacement = df_cpe_replacement1.union(df_cpe_replacement2).union(df_cpe_replacement3)
df_cpe_replacement = df_cpe_replacement.dropDuplicates()
df_cpe_replacement = df_cpe_replacement.na.drop(subset=['TICKET_ID','ASSET_ID','CPE_SERIAL_NUMBER'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 4.2 Loading Refurbishment data<a class="anchor" id="sec9"></a>

In [9]:
path_cpe_refurbishment = "hdfs://nameservicedev1//user/dt_srajan/predictive_care/replacement/CPE_refurbishment_2020_2021.csv"
df_cpe_refurbishment = spark.read.option("encoding", "ISO-8859-1").option("inferSchema", "true").csv(path_cpe_refurbishment,header = True,sep=",")

from pyspark.sql.types import *

df_cpe_refurbishment = df_cpe_refurbishment.withColumn('DATE_SCAN_END',F.when(F.unix_timestamp('Datum_izlaznog_skeniranja', "dd/MM/yyyy").cast(TimestampType()).isNotNull (),\
                                                                      F.unix_timestamp('Datum_izlaznog_skeniranja', "dd/MM/yyyy").cast(TimestampType()))\
                                                                .otherwise(None))\
                                            .withColumn('DATE_SCAN_START',F.when(F.unix_timestamp('Datum_ulaznog_skeniranja', "dd/MM/yyyy").cast(TimestampType()).isNotNull (),\
                                                                      F.unix_timestamp('Datum_ulaznog_skeniranja', "dd/MM/yyyy").cast(TimestampType()))\
                                                                .otherwise(None))

df_cpe_refurbishment = df_cpe_refurbishment.withColumn('DATE_SCAN_END',F.date_format(F.col("DATE_SCAN_END"), "yyyy-MM-dd HH:mm:ss"))\
                                            .withColumn('DATE_SCAN_START',F.date_format(F.col("DATE_SCAN_START"), "yyyy-MM-dd HH:mm:ss"))

df_cpe_refurbishment = df_cpe_refurbishment.withColumnRenamed('Vrsta kvara', 'Vrsta_kvara')

df_cpe_refurbishment.select('DATE_SCAN_START',  "DATE_SCAN_END").describe().show()

df_cpe_refurbishment = df_cpe_refurbishment.filter(df_cpe_refurbishment.Vrsta_kvara != 'NULL')
df_cpe_refurbishment = df_cpe_refurbishment.withColumn('Vrsta_kvara', F.regexp_replace('Vrsta_kvara', 'Napajanje', 'power_supply')) \
     .withColumn('Vrsta_kvara', F.regexp_replace('Vrsta_kvara', 'ZnaÄajno fiziÄko oÅ¡teÄenje - nepopravljivo', 'significant_physical_damage_irreparable')) \
     .withColumn('Vrsta_kvara', F.regexp_replace('Vrsta_kvara', 'Ne prijavljuje se u ACS', 'does_not_log_into_acs')) \
     .withColumn('Vrsta_kvara', F.regexp_replace('Vrsta_kvara', 'Reset ne radi', 'reset_not_work')) \
     .withColumn('Vrsta_kvara', F.regexp_replace('Vrsta_kvara', 'Manje fiziÄko ili toplinsko oÅ¡teÄenje', 'minor_physical_thermal_damage')) \
     .withColumn('Vrsta_kvara', F.regexp_replace('Vrsta_kvara', 'Ispravan', 'no_problem')) \
     .withColumn('Vrsta_kvara', F.regexp_replace('Vrsta_kvara', 'OptiÄki port', 'optical_port')) \
     .withColumn('Vrsta_kvara', F.regexp_replace('Vrsta_kvara', 'SIM utor', 'sim_slot')) \
     .withColumn('Vrsta_kvara', F.regexp_replace('Vrsta_kvara', 'WIFI', 'wifi')) \
     .withColumn('Vrsta_kvara', F.regexp_replace('Vrsta_kvara', 'Blokiran', 'blocked')) \
     .withColumn('Vrsta_kvara', F.regexp_replace('Vrsta_kvara', 'Resetira se sam', 'reset_itself')) \
     .withColumn('Vrsta_kvara', F.regexp_replace('Vrsta_kvara', 'Internet ne radi', 'internet_not_working')) \
     .withColumn('Vrsta_kvara', F.regexp_replace('Vrsta_kvara', 'DSL port', 'dsl_port')) \
     .withColumn('Vrsta_kvara', F.regexp_replace('Vrsta_kvara', 'Pregrijavanje ureÄaja', 'device_overheating')) \
     .withColumn('Vrsta_kvara', F.regexp_replace('Vrsta_kvara', 'LAN port', 'lan_port')) \
     .withColumn('Vrsta_kvara', F.regexp_replace('Vrsta_kvara', 'Slaba brzina', 'poor_speed')) \
     .withColumn('Vrsta_kvara', F.regexp_replace('Vrsta_kvara', 'Telefon port', 'telefon_port')) \
     .withColumn('Vrsta_kvara', F.regexp_replace('Vrsta_kvara', 'Software', 'software'))

df_cpe_refurbishment = df_cpe_refurbishment.withColumnRenamed('Vrsta_kvara', 'CPE_ISSUE')\
                                            .withColumnRenamed('Serijski_broj', 'CPE_SERIAL_NUMBER')
spaceDeleteUDF = F.udf(lambda s: s.replace("/ ", "/"), StringType())
df_cpe_refurbishment = df_cpe_refurbishment.withColumn("CPE_ISSUE", spaceDeleteUDF("CPE_ISSUE"))

df_cpe_refurbishment = df_cpe_refurbishment.select('CPE_SERIAL_NUMBER', 'DATE_SCAN_START','DATE_SCAN_END','CPE_ISSUE')
df_cpe_refurbishment.limit(3).show(200,truncate=False, vertical=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------------+-------------------+
|summary|    DATE_SCAN_START|      DATE_SCAN_END|
+-------+-------------------+-------------------+
|  count|              90920|              90920|
|   mean|               null|               null|
| stddev|               null|               null|
|    min|2020-01-02 00:00:00|2020-01-14 00:00:00|
|    max|2021-04-23 00:00:00|2021-04-30 00:00:00|
+-------+-------------------+-------------------+

-RECORD 0------------------------------------------
 CPE_SERIAL_NUMBER | j835bh004425                  
 DATE_SCAN_START   | 2020-01-14 00:00:00           
 DATE_SCAN_END     | 2020-01-14 00:00:00           
 CPE_ISSUE         | minor_physical_thermal_damage 
-RECORD 1------------------------------------------
 CPE_SERIAL_NUMBER | J833BH004104                  
 DATE_SCAN_START   | 2020-01-14 00:00:00           
 DATE_SCAN_END     | 2020-01-14 00:00:00           
 CPE_ISSUE         | minor_physical_thermal_damage 
-RECORD 2--------------------

### 4.3 Joining replacement and refurbishment data <a class="anchor" id="sec10"></a>

In [10]:
df = df_cpe_replacement.join(df_cpe_refurbishment, on = 'CPE_SERIAL_NUMBER', how = 'inner' )
df.groupBy("CPE_ISSUE") \
    .agg(F.countDistinct("ASSET_ID").alias("count_assetid")).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------+
|           CPE_ISSUE|count_assetid|
+--------------------+-------------+
|power_supply/lan_...|          128|
|          no_problem|        16605|
|minor_physical_th...|         1429|
|power_supply/dsl_...|          385|
|reset_itself/rese...|            1|
|telefon_port/powe...|           17|
|telefon_port/inte...|            2|
|                wifi|          187|
|does_not_log_into...|          206|
|reset_itself/dsl_...|            5|
|telefon_port/dsl_...|           16|
|telefon_port/lan_...|           15|
|internet_not_work...|            1|
|            software|            2|
|minor_physical_th...|            1|
|        power_supply|         4134|
|telefon_port/lan_...|            5|
|reset_itself/powe...|            1|
|            dsl_port|         4472|
|            lan_port|          301|
+--------------------+-------------+
only showing top 20 rows

### 4.4  Filtering CPEs with no_problem issue type <a class="anchor" id="sec11"></a>

In [11]:
df = df.filter(df.CPE_ISSUE != 'no_problem')
df.groupBy("CPE_ISSUE") \
    .agg(F.countDistinct("ASSET_ID").alias("count_assetid")).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------+
|           CPE_ISSUE|count_assetid|
+--------------------+-------------+
|power_supply/lan_...|          128|
|minor_physical_th...|         1429|
|power_supply/dsl_...|          385|
|reset_itself/rese...|            1|
|telefon_port/powe...|           17|
|telefon_port/inte...|            2|
|                wifi|          187|
|does_not_log_into...|          206|
|reset_itself/dsl_...|            5|
|telefon_port/dsl_...|           16|
|telefon_port/lan_...|           15|
|internet_not_work...|            1|
|            software|            2|
|minor_physical_th...|            1|
|        power_supply|         4134|
|telefon_port/lan_...|            5|
|reset_itself/powe...|            1|
|            dsl_port|         4472|
|            lan_port|          301|
|  device_overheating|          114|
+--------------------+-------------+
only showing top 20 rows

### 4.5  Filtering CPEs with some issues  <a class="anchor" id="sec12"></a>

We are filtering out the CPEs which are actually had any issues except "no_problem"

In [12]:
df_ticket_filtered_dsl = df_ticket_filtered_dsl.join(df,on = 'ASSET_ID', how = 'left_anti')
df_ticket_filtered_dsl = df_ticket_filtered_dsl.dropDuplicates()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
print('Total count:',df_ticket_filtered_dsl.count())
print('Unique CPEs:',df_ticket_filtered_dsl.select('ASSET_ID').distinct().count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

('Total count:', 645368)
('Unique CPEs:', 307036)

## 5. Filtering tickets data for DSL lines with all levels and exluding Thunderstorm cases  <a class="anchor" id="sec13"></a>

In [15]:
df_ticket_filtered_dsl_a = df_ticket_filtered_dsl.select('SOURCE','SOLUTION_LEVEL').dropDuplicates() 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
df_ticket_filtered_dsl_p = df_ticket_filtered_dsl_a.toPandas()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
df_ticket_filtered_dsl_p.groupby('SOURCE')['SOLUTION_LEVEL'].unique()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SOURCE
0915763509"                                                                  [nan]
098705584"                                                                   [nan]
?\t?Korisnik potvrdio da nema opasnosti za zdravlje tehnièara?"              [nan]
BBSA                                                                         [0.0]
DONAT                                                                        [1.0]
Ivana"                                                                       [nan]
KB 385922509978"                                                             [nan]
KB : 385955812189"                                                           [nan]
KB: 0959029975."                                                             [nan]
KB: 385917909536"                                                            [nan]
Korisnik potvrdio da nema opasnosti za zdravlje tehnièara - da"              [nan]
LP"                                                                          [na

**Insights:**
> Possible `Source` and `Solution Level` are:-

> When `Source` : BBSA possible `Solution Level` : 0
>
> When `Source` : DONAT possible `Solution Level` : 1
>
> When `Source` : WWMS possible `Solution Level` : [2,3,4]

<div class="alert alert-block alert-info">
<b>Check:</b> Let's check if same `TICKET_ID` raised multiple number of times for a particular `ASSET_ID`.
</div>

In [16]:
c = df_ticket_filtered_dsl.groupby('ASSET_ID','TICKET_ID').count().filter("'count'>'1'")
print(c.select(F.max('count')).show())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+
|max(count)|
+----------+
|         2|
+----------+

None

In [51]:
d = c.filter(F.col('count')>1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


<div class="alert alert-block alert-danger">
<b>Alert:</b> It shows a particluar cpes have same tickets raised maximum 2 times. So let's get the last ticket raised on same `ticket_id`.
</div>

In [17]:
df_ticket_filtered_dsl.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['ASSET_ID', 'TICKET_ID', 'TICKET_TENTATIVE_ROOT_CAUSE', 'WORK_CODE', 'Type_of_reported_interference', 'SOURCE', 'SOLUTION_LEVEL', 'SUMMARY', 'FAULT_RESOLUTION', 'deviation_mode', 'final_logout', 'START_DATE_TICKET']

In [18]:
path = '/user/tsystems_vkumar/tickets/df_all_tkts.parquet'
df_ticket_filtered_dsl.repartition(1).write.format('parquet').mode('overwrite').option('header','true').save(path)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
df_ticket_filtered_dsl.show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+---------+---------------------------+-------------+-----------------------------+------+--------------+-----------------+--------------------+--------------+--------------------+-------------------+
|ASSET_ID|TICKET_ID|TICKET_TENTATIVE_ROOT_CAUSE|    WORK_CODE|Type_of_reported_interference|SOURCE|SOLUTION_LEVEL|          SUMMARY|    FAULT_RESOLUTION|deviation_mode|        final_logout|  START_DATE_TICKET|
+--------+---------+---------------------------+-------------+-----------------------------+------+--------------+-----------------+--------------------+--------------+--------------------+-------------------+
|28513467|420084268|                       null|Nema podataka|                         null| DONAT|             1|    Nema podataka|       Nema podataka|          null|                null|2020-04-15 00:02:47|
|36648835| 44731364|               Blokada - 39|  FAULTREPAIR|                       prekid|  WWMS|             2|NEMA TEL. SIGNALA|Daljinski otklonj...|     Re

In [26]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window

w = Window.partitionBy(['ASSET_ID','TICKET_ID']).orderBy(desc('SOLUTION_LEVEL'),desc('START_DATE_TICKET'))
df_ticket_filtered_dsl_1 = df_ticket_filtered_dsl.withColumn('Rank',dense_rank().over(w))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
df_ticket_filtered_dsl_1.select(F.max('Rank')).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+
|max(Rank)|
+---------+
|        2|
+---------+

In [28]:
df_ticket_filtered_dsl_1 = df_ticket_filtered_dsl_1.filter(df_ticket_filtered_dsl_1.Rank==1)
df_ticket_filtered_dsl_1 = df_ticket_filtered_dsl_1.drop('Rank')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
path = '/user/tsystems_vkumar/tickets/df_all_tkts.parquet'
df_ticket_filtered_dsl.repartition(1).write.format('parquet').mode('overwrite').option('header','true').save(path)

> Get the final dataset for **tickets** and save it for future analysis

In [38]:
df_tickets = df_ticket_filtered_dsl_1.select(['ASSET_ID','START_DATE_TICKET','SOURCE','SOLUTION_LEVEL','SUMMARY','TICKET_TENTATIVE_ROOT_CAUSE','FAULT_RESOLUTION','deviation_mode','final_logout']).dropDuplicates()

#Remove the thunderstorm cases and save the data for further analysis
df_tickets = df_tickets.filter(F.col('TICKET_TENTATIVE_ROOT_CAUSE') != 'Grmljavina - 7')

#Select the columns
df_tickets = df_tickets.select(['ASSET_ID','START_DATE_TICKET','SOURCE','SOLUTION_LEVEL','SUMMARY','FAULT_RESOLUTION','deviation_mode','final_logout']).dropDuplicates()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
path = '/user/tsystems_vkumar/dsl_tickets/df_all_tkts_wo_thunder.parquet'
df_tickets.repartition(1).write.format('parquet').mode('overwrite').option('header','true').save(path)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
df_tickets.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-------------------+------+--------------+--------------------+--------------------+--------------+--------------------+
| ASSET_ID|  START_DATE_TICKET|SOURCE|SOLUTION_LEVEL|             SUMMARY|    FAULT_RESOLUTION|deviation_mode|        final_logout|
+---------+-------------------+------+--------------+--------------------+--------------------+--------------+--------------------+
| 45068665|2021-01-17 17:01:36| DONAT|             1|       Nema podataka|       Nema podataka|          null|                null|
| 28730371|2020-12-03 08:48:50|  WWMS|             2|    NEISPRAVAN MODEM|    Otkazana smetnja|     Nepoznato|Otkazana prijava ...|
| 44403229|2020-09-04 11:04:18| DONAT|             1|       Nema podataka|       Nema podataka|          null|                null|
|100873084|2020-06-02 18:43:25|  WWMS|             3|DSL SINHRONIZACIJ...|1.razina kod kori...|  Popravak - 1|Otklonjena smetnj...|
| 32871718|2021-01-02 22:26:24|  WWMS|             3|DSL SINHRONIZACIJ...|1.

In [6]:
df_tickets =  spark.read.option("header","true").parquet('hdfs://nameservicedev1////user/tsystems_vkumar/dsl_tickets/df_all_tkts_wo_thunder.parquet')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

 ### 5.1 Filter all DSL lines tickets irrespective of levels  <a class="anchor" id="sec14"></a>

Now for predicting issues related to DSL lines we need to do some filtering and need to consider all levels.

In [7]:
def get_specific_issue_data(df,summary):
    '''This Funtion filter the issue data as per the choice and exclude Thunderstorm cases
    
    Args:
        df (dataframe): Tickets dataset
        summary (str): Issue related to CPE
        
    Returns:
        df (dataframe) :  spark dataframe with filtered data.
    
    '''
    df = df.filter(F.col('SUMMARY').contains(summary))
    df = df.filter(df.SOURCE.isin(['BBSA','DONAT','WWMS']))
    df = df.dropDuplicates()
    
    cols = ['ASSET_ID','SOLUTION_LEVEL','START_DATE_TICKET']
    df = df.select(['ASSET_ID','SOLUTION_LEVEL','START_DATE_TICKET','FAULT_RESOLUTION','deviation_mode','final_logout']).distinct()
    df = df.orderBy(*cols, ascending=False)
    return df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<div class="alert alert-block alert-info">
<b>Tip:</b> We will be analysing on all th raised issues and check the 'Solutions' provided.
</div>

In [8]:
df_ticket_filtered_dsl_f = get_specific_issue_data(df_tickets,summary='DSL SINHRONIZACIJ')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
print(df_ticket_filtered_dsl_f.count())
df_ticket_filtered_dsl_f.select('ASSET_ID').distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

146123
108400

In [41]:
print(df_ticket_filtered_dsl_f.count())
df_ticket_filtered_dsl_f.select('ASSET_ID').distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

146123
108400

That means each `asset_id` has raised tickets on multiple levels. And our analysis should be on to capture the tickets raised by any cpe at their maximum level.

<div class="alert alert-block alert-success">
<b>Filter:</b> Let's check the resoltion provided for the different `level`
</div>

In [22]:
a = list(df_ticket_filtered_dsl_fp.final_logout.unique())
a

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[u'Po ispitivanju dobar (gre\x9aka nije prona\xf0ena) - 2', u'Otklonjena smetnja - prekid - 1', u'Otklonjena smetnja - degradacija 45', u'Usluga ne zadovoljava o\xe8ekivanja korisnika (korisnik upu\xe6en) - 31', u'Otkazana prijava (korisnik odjavio smetnju ili krivo prijavljena smetnja) - 30', u'Nepoznato', u'Smetnja nije otklonjena - 35', u'Privremeno otklonjena smetnja - 3', u'Isklju\xe8enja zbog duga ili na zahtjev korisnika - 32', u'Automatskom dijagnostikom nije utvr\xf0eno postojanje smetnje']

In [23]:
for i,element in enumerate(a):
    print(element)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Po ispitivanju dobar (greka nije pronaðena) - 2
Otklonjena smetnja - prekid - 1
Otklonjena smetnja - degradacija 45
Usluga ne zadovoljava oèekivanja korisnika (korisnik upuæen) - 31
Otkazana prijava (korisnik odjavio smetnju ili krivo prijavljena smetnja) - 30
Nepoznato
Smetnja nije otklonjena - 35
Privremeno otklonjena smetnja - 3
Iskljuèenja zbog duga ili na zahtjev korisnika - 32
Automatskom dijagnostikom nije utvrðeno postojanje smetnje

In [10]:
mapping_1 = { '1.razina kod korisnika' : '1st level with user',
    'Razina na agregaciji': 'Level on aggregation',
            '2.razina na mrei': '2nd level online',
                '1.razina na mrei': '1st level online',
                'Daljinski otklonjena smetnja': 'Remote interference',
                'Otkazana smetnja': 'Canceled interference',
            'Nepoznato':'Unknown',
            'Drugi operator': 'Other operator',
                'Automatska dijagnostika': 'Automatic diagnostics'}

mapping_2 = {'Nepoznato':'Unknown',
'Promjena profila - 2': 'Change profile - 2',
'Zamjena - 8': 'Replacement - 8',
'Rekonfiguracija - 3': 'Reconfiguration - 3',
'Popravak - 1': 'Repair - 1',
'Reset - 4': 'Reset - 4',
'Vlastito odravanje korisnika - 6': 'Own user maintenance - 6',
'Onemoguæen pristup prostoru u vlasnitvu korisnika - 5': 'Disabled access to user-owned space - 5',
'Drugi operator - 9': 'Second operator - 9'}

mapping_3 = { 'Po ispitivanju dobar (greka nije pronaðena) - 2': 'After testing good (error not found) - 2',
'Otklonjena smetnja - prekid - 1': 'Fault rectified - interrupt - 1',
'Otklonjena smetnja - degradacija 45': 'Troubleshooting - degradation 45',
'Usluga ne zadovoljava oèekivanja korisnika (korisnik upuæen) - 31': 'Service does not meet user expectations (user referred) - 31',
'Otkazana prijava (korisnik odjavio smetnju ili krivo prijavljena smetnja) - 30': 'Canceled login (user checked out the fault or incorrectly reported the fault) - 30',
'Nepoznato': 'Unknown',
'Smetnja nije otklonjena - 35': 'Troubleshooting - 35',
'Privremeno otklonjena smetnja - 3': 'Temporarily removed interference - 3',
'Iskljuèenja zbog duga ili na zahtjev korisnika - 32': 'Exclusions due to debt or at the request of the user - 32',
'Automatskom dijagnostikom nije utvrðeno postojanje smetnje': 'Self-diagnosis does not detect the presence of a fault'}

df_ticket_filtered_dsl_f = df_ticket_filtered_dsl_f.replace(to_replace=mapping_1,subset=['FAULT_RESOLUTION'])
df_ticket_filtered_dsl_f = df_ticket_filtered_dsl_f.replace(to_replace=mapping_2,subset=['deviation_mode'])
df_ticket_filtered_dsl_f = df_ticket_filtered_dsl_f.replace(to_replace=mapping_3,subset=['final_logout'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
df_ticket_filtered_dsl_f.select(F.col('FAULT_RESOLUTION')).distinct().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|    FAULT_RESOLUTION|
+--------------------+
|Automatic diagnos...|
|    2nd level online|
| Remote interference|
|      Other operator|
|             Unknown|
| 1st level with user|
|    1st level online|
|Canceled interfer...|
|Level on aggregation|
+--------------------+

In [27]:
df_ticket_filtered_dsl_f.select(F.col('deviation_mode')).distinct().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|      deviation_mode|
+--------------------+
|Disabled access t...|
|     Replacement - 8|
|             Unknown|
| Reconfiguration - 3|
| Second operator - 9|
|Own user maintena...|
|  Change profile - 2|
|          Repair - 1|
|           Reset - 4|
+--------------------+

In [28]:
df_ticket_filtered_dsl_f.select(F.col('final_logout')).distinct().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|        final_logout|
+--------------------+
|Exclusions due to...|
|Fault rectified -...|
|             Unknown|
|Canceled login (u...|
|Troubleshooting -...|
|Service does not ...|
|After testing goo...|
|Temporarily remov...|
|Self-diagnosis do...|
|Troubleshooting - 35|
+--------------------+

In [11]:
df_ticket_filtered_dsl_f.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------+-------------------+--------------------+-------------------+--------------------+
|ASSET_ID|SOLUTION_LEVEL|  START_DATE_TICKET|    FAULT_RESOLUTION|     deviation_mode|        final_logout|
+--------+--------------+-------------------+--------------------+-------------------+--------------------+
|99999823|             3|2020-02-17 14:34:16|    1st level online|    Replacement - 8|Fault rectified -...|
|99999615|             2|2020-08-25 11:13:06| Remote interference|Reconfiguration - 3|Fault rectified -...|
|99999432|             2|2020-03-24 08:51:38|Level on aggregation|          Reset - 4|Fault rectified -...|
|99997653|             2|2020-07-11 21:22:39|Level on aggregation|          Reset - 4|Fault rectified -...|
|99997653|             2|2020-06-03 18:35:17|Level on aggregation|          Reset - 4|Troubleshooting -...|
+--------+--------------+-------------------+--------------------+-------------------+--------------------+
only showing top 5 rows

In [12]:
path = '/user/tsystems_vkumar/dsl_tickets/filtered_cpe_dsl_all_lvl.parquet'
df_ticket_filtered_dsl_f.repartition(1).write.format('parquet').mode('overwrite').option('header','true').save(path)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
#df_ticket_filtered_dsl_f =  spark.read.option("header","true").parquet('hdfs://nameservicedev1////user/tsystems_vkumar/dsl_tickets/filtered_cpe_dsl_all_lvl.parquet')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
df_ticket_filtered_dsl_fp = df_ticket_filtered_dsl_f.toPandas()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
(df_ticket_filtered_dsl_fp.groupby('SOLUTION_LEVEL')['deviation_mode'].value_counts()/df_ticket_filtered_dsl_fp.groupby('SOLUTION_LEVEL')['deviation_mode'].count())*100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SOLUTION_LEVEL  deviation_mode                         
2               Unknown                                    36.950803
                Reset - 4                                  31.697358
                Repair - 1                                 15.061447
                Reconfiguration - 3                        12.599631
                Change profile - 2                          2.120225
                Own user maintenance - 6                    0.993364
                Replacement - 8                             0.545762
                Disabled access to user-owned space - 5     0.031411
3               Repair - 1                                 43.164225
                Replacement - 8                            24.770751
                Unknown                                    12.052916
                Own user maintenance - 6                    8.381962
                Reset - 4                                   5.568457
                Reconfiguration - 3            

In [15]:
(df_ticket_filtered_dsl_fp.groupby('SOLUTION_LEVEL')['FAULT_RESOLUTION'].value_counts()/df_ticket_filtered_dsl_fp.groupby('SOLUTION_LEVEL')['FAULT_RESOLUTION'].count())*100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SOLUTION_LEVEL  FAULT_RESOLUTION     
2               Remote interference       53.539597
                Level on aggregation      36.208724
                Canceled interference      9.340767
                1st level with user        0.577172
                1st level online           0.325886
                Automatic diagnostics      0.003926
                Unknown                    0.003926
3               1st level with user       53.455599
                1st level online          33.806717
                Level on aggregation       6.913183
                Canceled interference      3.020920
                Remote interference        1.486642
                Unknown                    1.199833
                Other operator             0.117105
4               2nd level online         100.000000
Name: FAULT_RESOLUTION, dtype: float64

In [16]:
(df_ticket_filtered_dsl_fp.groupby('SOLUTION_LEVEL')['final_logout'].value_counts()/df_ticket_filtered_dsl_fp.groupby('SOLUTION_LEVEL')['final_logout'].count())*100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SOLUTION_LEVEL  final_logout                                                                      
2               Fault rectified - interrupt - 1                                                       56.876988
                Troubleshooting - degradation 45                                                      29.109898
                Canceled login (user checked out the fault or incorrectly reported the fault) - 30     9.340767
                After testing good (error not found) - 2                                               4.142291
                Troubleshooting - 35                                                                   0.168833
                Unknown                                                                                0.125643
                Exclusions due to debt or at the request of the user - 32                              0.117790
                Service does not meet user expectations (user referred) - 31                           0.070674
     

> It has been seen that a particular `asset_id` raised issue later also in higher `solution_level` to get fix their DSL lines. So it makes sense to take the highest `solution_level` reached for them.

<div class="alert alert-block alert-danger">
<b>Must:</b> Prepare the data with per asset id we see their tickets reached at the maximum level at any period of time.
</div>

In [18]:
df_ticket_filtered_dsl_f1 = df_ticket_filtered_dsl_f.select('ASSET_ID','SOLUTION_LEVEL','START_DATE_TICKET').distinct()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
df_ticket_filtered_dsl_f1 = df_ticket_filtered_dsl_f1.na.drop(subset=['START_DATE_TICKET'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
df_ticket_filtered_dsl_f2 = df_ticket_filtered_dsl_f1.groupby('ASSET_ID').agg(F.max('SOLUTION_LEVEL').alias('SOLUTION_LEVEL'))
df_ticket_filtered_dsl_f2 = df_ticket_filtered_dsl_f2.join(df_ticket_filtered_dsl_f1,on=['ASSET_ID','SOLUTION_LEVEL'],how='left')
df_ticket_filtered_dsl_f2 = df_ticket_filtered_dsl_f2.groupby(['ASSET_ID','SOLUTION_LEVEL']).agg(F.max('START_DATE_TICKET').alias('START_DATE_TICKET'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
df_ticket_filtered_dsl_f2.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ASSET_ID: string (nullable = true)
 |-- SOLUTION_LEVEL: integer (nullable = true)
 |-- START_DATE_TICKET: string (nullable = true)

In [60]:
df_ticket_filtered_dsl_f2.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------+-------------------+
| ASSET_ID|SOLUTION_LEVEL|  START_DATE_TICKET|
+---------+--------------+-------------------+
|100065331|             3|2020-11-25 20:16:43|
|100942825|             3|2021-04-09 09:14:53|
|101203934|             3|2021-03-30 21:07:14|
|101519291|             3|2020-11-20 12:35:59|
|101852767|             4|2021-03-04 08:42:29|
+---------+--------------+-------------------+
only showing top 5 rows

In [24]:
print(df_ticket_filtered_dsl_f2.count())
df_ticket_filtered_dsl_f2.select('ASSET_ID').distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

107307
107307

In [22]:
df_ticket_filtered_dsl_f2p = df_ticket_filtered_dsl_f2.toPandas

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<div class="alert alert-block alert-info">
<b>Distribution:</b> Distribution of Tickets per level
</div>

In [30]:
(df_ticket_filtered_dsl_f2p.SOLUTION_LEVEL.value_counts()/df_ticket_filtered_dsl_f2p.shape[0])*100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3    68.762522
4    16.425769
2    14.811708
Name: SOLUTION_LEVEL, dtype: float64

In [25]:
df_ticket_filtered_dsl_f2p1 = df_ticket_filtered_dsl_f2p.merge(df_ticket_filtered_dsl_fp,on=['ASSET_ID','SOLUTION_LEVEL','START_DATE_TICKET'],how='inner')
print(df_ticket_filtered_dsl_f2p1.shape)
print(df_ticket_filtered_dsl_f2p1.ASSET_ID.nunique())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(107573, 6)
107307

In [27]:
(df_ticket_filtered_dsl_f2p1.groupby('SOLUTION_LEVEL')['deviation_mode'].value_counts()/df_ticket_filtered_dsl_fp.groupby('SOLUTION_LEVEL')['deviation_mode'].count())*100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SOLUTION_LEVEL  deviation_mode                         
2               Unknown                                    23.082964
                Reset - 4                                  19.600298
                Repair - 1                                  9.482116
                Reconfiguration - 3                         7.935137
                Change profile - 2                          1.346735
                Own user maintenance - 6                    0.632141
                Replacement - 8                             0.337665
                Disabled access to user-owned space - 5     0.015705
3               Repair - 1                                 32.519551
                Replacement - 8                            18.359732
                Unknown                                     8.415704
                Own user maintenance - 6                    6.018022
                Reset - 4                                   3.899210
                Reconfiguration - 3            

In [28]:
(df_ticket_filtered_dsl_f2p1.groupby('SOLUTION_LEVEL')['FAULT_RESOLUTION'].value_counts()/df_ticket_filtered_dsl_fp.groupby('SOLUTION_LEVEL')['FAULT_RESOLUTION'].count())*100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SOLUTION_LEVEL  FAULT_RESOLUTION     
2               Remote interference      32.847776
                Level on aggregation     23.067258
                Canceled interference     5.968040
                1st level with user       0.325886
                1st level online          0.219875
                Automatic diagnostics     0.003926
3               1st level with user      39.337462
                1st level online         24.979159
                Level on aggregation      4.982930
                Canceled interference     2.096979
                Remote interference       1.037077
                Unknown                   0.907070
                Other operator            0.083363
4               2nd level online         88.924082
Name: FAULT_RESOLUTION, dtype: float64

In [29]:
(df_ticket_filtered_dsl_f2p1.groupby('SOLUTION_LEVEL')['final_logout'].value_counts()/df_ticket_filtered_dsl_fp.groupby('SOLUTION_LEVEL')['final_logout'].count())*100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SOLUTION_LEVEL  final_logout                                                                      
2               Fault rectified - interrupt - 1                                                       35.914249
                Troubleshooting - degradation 45                                                      17.955161
                Canceled login (user checked out the fault or incorrectly reported the fault) - 30     5.968040
                After testing good (error not found) - 2                                               2.281205
                Troubleshooting - 35                                                                   0.109938
                Unknown                                                                                0.090306
                Exclusions due to debt or at the request of the user - 32                              0.054969
                Temporarily removed interference - 3                                                   0.031411
     

> Now we hav the list of `assetid` which have the `START_DATE_TICKET` reached to the maximum level. However, we should also verify that all `assetid` are available in our inventory database(common overlapping period for all databases in the given period).

> So let's get the list of all `assetid` which are present in the inventory.

In [32]:
df_ticket_filtered_dsl_f2 = df_ticket_filtered_dsl_f2.withColumn('label',F.lit('DSL synchronization'))
df_ticket_filtered_dsl_f2 = df_ticket_filtered_dsl_f2.select(F.col('ASSET_ID').alias('assetid')\
                                                             ,F.col('START_DATE_TICKET').alias('incident_date')\
                                                             ,F.col('label').alias('label')\
                                                             ,F.col('SOLUTION_LEVEL').alias('level'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<div class="alert alert-block alert-warning">
<b>Filter</b> After having the asset ids with tickets at their maximum levels. We need to filter only thoses asset ids which are there in the inventory. 
</div>

In [33]:
df_tkts_inventory =  spark.read.option("header","true").parquet('hdfs://nameservicedev1//user/dt_srajan/inventory_filtered_with_tickets')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
df_ticket_filtered_dsl_f3 = df_ticket_filtered_dsl_f2.join(df_tkts_inventory,on='assetid',how='inner')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [69]:
df_ticket_filtered_dsl_f3.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-------------------+-------------------+-----+
|  assetid|      incident_date|              label|level|
+---------+-------------------+-------------------+-----+
|100065331|2020-11-25 20:16:43|DSL synchronization|    3|
|100942825|2021-04-09 09:14:53|DSL synchronization|    3|
|101203934|2021-03-30 21:07:14|DSL synchronization|    3|
|101852767|2021-03-04 08:42:29|DSL synchronization|    4|
|101858212|2021-01-14 15:10:41|DSL synchronization|    3|
+---------+-------------------+-------------------+-----+
only showing top 5 rows

In [35]:
print(df_ticket_filtered_dsl_f3.count())
df_ticket_filtered_dsl_f3.select('assetid').distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

104058
104058

In [71]:
print('Level 0 tickets:',df_ticket_filtered_dsl_f3.filter(F.col('level')==0).count())
print('\n')
print('Level 1 tickets:',df_ticket_filtered_dsl_f3.filter(F.col('level')==1).count())
print('\n')
print('Level 2 tickets:',df_ticket_filtered_dsl_f3.filter(F.col('level')==2).count())
print('\n')
print('Level 3 tickets:',df_ticket_filtered_dsl_f3.filter(F.col('level')==3).count())
print('\n')
print('Level 4 tickets:',df_ticket_filtered_dsl_f3.filter(F.col('level')==4).count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

('Level 0 tickets:', 0)


('Level 1 tickets:', 0)


('Level 2 tickets:', 15382)


('Level 3 tickets:', 71589)


('Level 4 tickets:', 17087)

In [39]:
path = '/user/tsystems_vkumar/dsl_tickets/dsl_tkts_all_lvls_uniq_n.parquet'
df_ticket_filtered_dsl_f3.repartition(1).write.format('parquet').mode('overwrite').option('header','true').save(path)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
df_ticket_filtered_dsl_f4 = df_ticket_filtered_dsl_f3.select('assetid','incident_date','label')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
path = '/user/tsystems_vkumar/dsl_tickets/dsl_tkts_all_lvls_uniq_wolbl.parquet'
df_ticket_filtered_dsl_f4.repartition(1).write.format('parquet').mode('overwrite').option('header','true').save(path)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 6. Filter Non DSL tickets for healthy dataset <a class="anchor" id="sec15"></a>

Load the created tickets dataset with all issues except Thunderstorm cases.

In [43]:
df_tickets =  spark.read.option("header","true").parquet('hdfs://nameservicedev1////user/tsystems_vkumar/dsl_tickets/df_all_tkts_wo_thunder.parquet')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<div class="alert alert-block alert-success">
<b>Conditions:</b> We need to consider Non-DSL tickets which can be included in our healthy dataset to add noise. So after shortlisting below are the non-DSL issues which have been included.
</div>



In [44]:
#df_tickets_dth = df_tickets.filter(F.col("SUMMARY").contains("DTH"))
df_tickets_radi = df_tickets.filter(F.col("SUMMARY")=='BEIÈNA VEZA NE RADI')
df_tickets_a = df_tickets.filter(F.col("SUMMARY")=='STALNO ZAUZET')
df_tickets_b = df_tickets.filter(F.col("SUMMARY")=='NE RADE DOLAZNI NI ODLAZNI POZIVI (VOIP)')
df_tickets_c = df_tickets.filter(F.col("SUMMARY")=='DODATNE USLUGE')
df_tickets_d = df_tickets.filter(F.col("SUMMARY")=='2. STB SE NE BOOTA')
df_tickets_e = df_tickets.filter(F.col("SUMMARY")=='NEISPRAVAN STB')
df_tickets_f = df_tickets.filter(F.col("SUMMARY")=='STB SE POVREMENO GASI')
df_tickets_g = df_tickets.filter(F.col("SUMMARY")=='NEISPRAVAN DALJINSKI UPRAVLJAÈ')
df_tickets_h = df_tickets.filter(F.col("SUMMARY")=='SMETNJA NA VIDEOTECI')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
#df_tkts_comb = df_tickets_dth.union(df_tickets_radi).union(df_tickets_a).union(df_tickets_c)\
 #                            .union(df_tickets_d).union(df_tickets_e).union(df_tickets_f).union(df_tickets_g)\
  #                              .union(df_tickets_h)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
df_tkts_comb = df_tickets_radi.union(df_tickets_a).union(df_tickets_c)\
                             .union(df_tickets_d).union(df_tickets_e).union(df_tickets_f).union(df_tickets_g)\
                                .union(df_tickets_h)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
df_tkts_comb.select(F.col('SUMMARY')).distinct().toPandas().SUMMARY.unique()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

array([u'2. STB SE NE BOOTA', u'NEISPRAVAN STB', u'STALNO ZAUZET',
       u'BE\x8eI\xc8NA VEZA NE RADI', u'SMETNJA NA VIDEOTECI',
       u'NEISPRAVAN DALJINSKI UPRAVLJA\xc8', u'DODATNE USLUGE',
       u'STB SE POVREMENO GASI'], dtype=object)

In [48]:
df_tkts_comb = df_tkts_comb.dropDuplicates()
print(df_tkts_comb.count())
df_tkts_comb.select(F.col('ASSET_ID')).distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

51019
44899

In [26]:
df_tkts_comb = spark.read.option("header","true").parquet('hdfs://nameservicedev1///user/tsystems_vkumar/dsl_tickets/df_tkts_non_dsl.parquet')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

We need to take the lastest issue recorded for all the filtered issue type

In [49]:
df_tkts_comb_agg_1 = df_tkts_comb.groupby('ASSET_ID','SUMMARY').agg(F.max('START_DATE_TICKET').alias('START_DATE_TICKET'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
print(df_tkts_comb_agg_1.count())
df_tkts_comb_agg_1.select(F.col('ASSET_ID')).distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

47540
44899

In [51]:
#Check null count
df_tkts_comb_agg_1.select([F.count(F.when(F.isnan(c) | col(c).isNull(), c)).alias(c) for c in df_tkts_comb_agg_1.columns]).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+-----------------+
|ASSET_ID|SUMMARY|START_DATE_TICKET|
+--------+-------+-----------------+
|       0|      0|              560|
+--------+-------+-----------------+

In [52]:
df_tkts_comb_agg_1 = df_tkts_comb_agg_1.na.drop()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
df_tkts_comb_agg = df_tkts_comb_agg_1.select(F.col('ASSET_ID').alias('assetid'),F.col('START_DATE_TICKET').alias('incident_date'),F.col('SUMMARY').alias('label'))
df_tkts_comb_agg.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-------------------+--------------------+
|  assetid|      incident_date|               label|
+---------+-------------------+--------------------+
|100424784|2021-03-26 16:03:55|       STALNO ZAUZET|
|101866361|2020-04-04 20:13:56|       STALNO ZAUZET|
|102594887|2020-09-22 14:33:38|       STALNO ZAUZET|
|103232787|2020-11-27 08:09:41|NEISPRAVAN DALJIN...|
|103318378|2021-01-18 17:31:26|BEIÈNA VEZA NE RADI|
+---------+-------------------+--------------------+
only showing top 5 rows

In [54]:
df_tkts_comb_agg_1 = df_tkts_comb_agg.drop('label')
df_tkts_comb_agg_1 = df_tkts_comb_agg.withColumn('label',F.lit('Non-DSL issue'))
df_tkts_comb_agg_1 = df_tkts_comb_agg_1.dropDuplicates()
df_tkts_comb_agg_1.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------------------+-------------+
| assetid|      incident_date|        label|
+--------+-------------------+-------------+
|39919302|2021-02-09 13:33:12|Non-DSL issue|
|49008735|2020-01-18 14:25:24|Non-DSL issue|
|47177519|2020-09-09 14:17:01|Non-DSL issue|
|61693123|2020-06-01 10:51:53|Non-DSL issue|
|83961484|2020-02-20 15:57:03|Non-DSL issue|
+--------+-------------------+-------------+
only showing top 5 rows

<div class="alert alert-block alert-warning">
<b>Filter</b> After having the asset ids with Non-DSL tickets. We need to filter only thoses asset ids which are there in the inventory.
</div>

In [55]:
df_tkts_comb_agg_2 = df_tkts_comb_agg_1.join(df_tkts_inventory,on='assetid',how='inner')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
#df_tkts_comb_agg_1 = df_tkts_comb_agg_1.withColumn('incident_date',F.col('incident_date').cast('date'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
df_tkts_comb_agg_2 = df_tkts_comb_agg_2.dropDuplicates()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [58]:
print(df_tkts_comb_agg_2.count())
df_tkts_comb_agg_2.select(F.col('assetid')).distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

46097
43530

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window

w = Window.partitionBy(['assetid']).orderBy(desc('incident_date')
df_tkts_comb_agg_2 = df_tkts_comb_agg_2.withColumn('Rank',dense_rank().over(w))
df_tkts_comb_agg_2 = df_tkts_comb_agg_2.fiter(df_tkts_comb_agg_2.Rank==1)  
df_tkts_comb_agg_2.drop(subset=['Rank'])

In [59]:
path = '/user/tsystems_vkumar/non_dsl_tickets/non_dsl_tkts_n.parquet'
df_tkts_comb_agg_2.repartition(1).write.format('parquet').mode('overwrite').option('header','true').save(path)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Not required  but it's for translation

In [21]:
mapping = {'NEISPRAVAN MODEM - TEHNIÈAR':'DEFECTIVE MODEM - TECHNICIAN',
'OPTIKA - NEISPRAVAN ONT':'OPTICS - DEFECTIVE ONT',
'DSL SINHRONIZIRAN: POVREMENI PREKIDI KONEKCIJE':'DSL SYNCHRONIZED, OCCASIONAL INTERRUPTIONS',
'NEMA TEL. SIGNALA':'NO TEL. SIGNAL',
'NEISPRAVAN MODEM':'DEFECTIVE MODEM',
'STALNO ZAUZET':'CONSTANTLY OCCUPIED',
'NE PROLAZI BOOT PROCEDURA':'DOES NOT PASS THE BOOT PROCEDURE',
'MAXNET MINI':'MAXNET MINI',
'KORISNIÈKA OPREMA':'USER EQUIPMENT',
'DSL SINHRONIZACIJA - PREKID':'DSL SYNCHRONIZATION - INTERRUPTION',
'POTEKOÆA S BRZINOM':'DIFFICULTY WITH SPEED',
'NE RADE DOLAZNI NI ODLAZNI POZIVI (VOIP)':'INBOUT OR OUTFLOW CALLS (VOIP) DO NOT WORK',
'DSL SINHRONIZACIJA - POVREMENI PREKIDI':'DSL SYNCHRONIZATION - OCCASIONAL INTERRUPTIONS',
'DTH - POVREMENO SE GUBI SATELITSKI SIGNAL':'DTH - OCCASIONALLY LOSS SATELLITE SIGNAL',
'NEMOGUÆE UTVRDITI':'IMPOSSIBLE TO DETERMINE',
'BEIÈNA VEZA NE RADI':'WIRELESS DOES NOT WORK',
'MODEM NEISPRAVAN (GRMLJAVINA)':'MODEM FAULT (THUNDERSTORM)',
'NEISPRAVAN MODEM (GRMLJAVINA)':'DEFECTIVE MODEM (THUNDERSTORM)',
'NE RADE DOLAZNI POZIVI':'INCOMING CALLS DO NOT WORK',
'MODEM NEISPRAVAN (GRMLJAVINA) - TEHNIÈAR':'MODEM FAULT (THUNDERSTORM) - TECHNICIAN',
'DODATNE USLUGE':'ADDITIONAL SERVICES',
'DTH - NEMA SATELITSKOG SIGNALA':'DTH - NO SATELLITE SIGNAL',
'NE RADE DOLAZNI NI ODLAZNI POZIVI':'INCOMING OR OUTGOING CALLS DO NOT WORK',
'NEISPRAVAN IP TELEFON':'DEFECTIVE IP PHONE',
'DTH - NEUSPJENO UPARIVANJE':'DTH - PAIR FAILURE',
'NEISPRAVAN MODEM (VoIP) - TEHNIÈAR':'DEFECTIVE MODEM (VoIP) - TECHNICIAN',
'2. STB SE NE BOOTA':'2. STB SE NE BOOTA',
'NEISPRAVAN STB':'DEFECTIVE STB',
'NEISPRAVAN MODEM (GRMLJAVINA) - TEHNIÈAR':'DEFECTIVE MODEM (THUNDERSTORM) - TECHNICIAN',
'STB SE POVREMENO GASI':'STB IS OCCASIONALLY EXTINGUISHED',
'USER SPOJEN: NE OTVARA STRANICE':'USER CONNECTED, DOES NOT OPEN PAGES',
'ZAMRZAVANJE SLIKE':'PICTURE FREEZING',
'NEISPRAVAN DALJINSKI UPRAVLJAÈ':'DEFECTIVE REMOTE CONTROL',
'NE RADI SWITCH':'SWITCH DOES NOT WORK',
'OPTIKA - POVREMENI PREKIDI /NEMA OPTIÈKOG LINKA':'OPTICS - OCCASIONAL INTERRUPTIONS / NO OPTICAL LINK',
'SMETNJA NA SNIMALICI':'INTERFERENCE ON THE RECORDER',
'DSL SINHRONIZIRAN: USER SE NE SPAJA':'DSL SYNCHRONIZED, USER DOES NOT CONNECT',
'NE VIDE SE POJEDINI KANALI':'INDIVIDUAL CHANNELS CANNOT BE SEEN',
'OPTIKA - NEMA AKTIVACIJE MODEMA':'OPTICS - NO MODEM ACTIVATION',
'NEISPRAVAN STB (GRMLJAVINA)':'DEFECTIVE STB (THUNDERSTORM)',
'NEMA ZVUKA':'NO SOUND',
'DTH - NEISPRAVNA DEKODERSKA KARTICA':'DTH - DEFECTIVE DECODER CARD',
'GREKA NA KUÆNOJ INSTALACIJI':'HOUSEHOLD INSTALLATION ERROR',
'NEISPRAVAN MODEM (VoIP)':'FAULT MODEM (VoIP)',
'NE RADE SAMO DOLAZNI POZIVI':'ONLY INCOMING CALLS DO NOT WORK',
'NEISPRAVAN MEDIATRIX':'DEFECTIVE MEDIATRIX',
'PROBLEMI S DALJINSKIM UPRAVLJAÈEM':'REMOTE CONTROL PROBLEMS',
'OPTIKA - OTEÆENA KUÆNA OPTIÈKA INSTALACIJA':'OPTICS - DAMAGED HOME OPTICAL INSTALLATION',
'OPTIKA - NEISPRAVAN ONT (GRMLJAVINA)':'OPTICS - DEFECTIVE ONT (THUNDER)',
'IÈNA VEZA MODEM  - PC NE RADI':'WIRE CONNECTION MODEM - PC DOES NOT WORK',
'TRZANJE SLIKE':'PICTURE PICTURE',
'SLABA ÈUJNOST':'POOR HEARING',
'NE RADE SAMO ODLAZNI POZIVI':'ONLY OUTSTANDING CALLS DO NOT WORK',
'DTH - novi transponder':'DTH - new transponder',
'Prekid HA usluge':'Termination of HA service',
'Prekid usluga i prekid 4G backup-a':'Interruption of services and termination of 4G backup',
'PREKID VEZE KOD JAVLJANJA':'DISCONNECTION AT THE REPORT',
'DTH - OTEÆEN ANTENSKI SUSTAV':'DTH - DAMAGED ANTENNA SYSTEM',
'SMETNJE U TOKU VEZE (UM)':'INTERFERENCE DURING CONNECTION (NOISE)',
'NE RADI - JTG':'NOT WORKING - JTG',
'STALNO ZVONI':'CONSTANTLY RINGS',
'SMETNJA NA USLUZI SIGURAN DOM':'INTERFERENCE WITH SAFE HOME SERVICE',
'OPTIKA - IMA LINKA - NEMA AKTIVACIJE ONT-a':'OPTIKA - THERE IS A LINK - NO ONT ACTIVATION',
'SMETNJE U TOKU VEZE (JEKA:KANJENJE GLASA)':'INTERFERENCE DURING THE COMMUNICATION (Echo, VOICE DELAY)',
'TeraStream ? prekid':'TeraStream? break',
'DTH - NEMA ZEMALJSKIH KANALA':'DTH - NO EARTH CHANNELS',
'SMETNJA NA VIDEOTECI':'INTERFERENCE AT THE VIDEO LIBRARY',
'DTH - KANAL KODIRAN':'DTH - CHANNEL CHANNEL',
'PRESLUAVANJE U TOKU RAZGOVORA':'LISTENING DURING THE INTERVIEW',
'ZAMJENA BROJEVA':'NUMBER REPLACEMENT',
'NE RADE ODLAZNI POZIVI':'DETAILED CALLS DO NOT WORK',
'Multi Office - POTEKOÆA KOD PRISTUPA LOKACIJAMA':'Multi Office - DIFFICULTIES IN ACCESSING LOCATIONS',
'JEDNOSTRANA ÈUJNOST':'UNILATERAL HEARING',
'NE RADI INTERNET (DIAL - UP)':'THE INTERNET DOES NOT WORK (DIAL - UP)',
'MaxTv2Go':'MaxTv2Go',
'HotSpotFon':'HotSpotFon',
'NE RADE DODATNE OPCIJE (TXT: WIDGET: EPG)':'ADDITIONAL OPTIONS (TXT, WIDGET, EPG) DO NOT WORK',
'POS APARAT NE RADI':'POS APPLIANCE DOES NOT WORK',
'NEISPRAVAN MODEM - TCENTAR':'DEFECTIVE MODEM - TCENTAR',
'Nema podataka':'No data',
'MODEM NEISPRAVAN (GRMLJAVINA) - DISTRIBUTER':'MODEM DEFECTIVE (THUNDERSTORM) - DISTRIBUTOR',
'NEMA REGISTRACIJE BROJA':'NO NUMBER REGISTRATION',
'NEISPRAVAN MODEM - DISTRIBUTER':'DEFECTIVE MODEM - DISTRIBUTOR',
'MODEM NEISPRAVAN (GRMLJAVINA) - TCENTAR':'MODEM DEFECTIVE (THUNDERSTORM) - TCENTAR',
'SPLITER NEISPRAVAN':'SPLITTER DEFECTIVE',
'NEISPRAVAN MODEM (GRMLJAVINA) - DISTRIBUTER':'DEFECTIVE MODEM (THUNDERSTORM) - DISTRIBUTOR',
'NEISPRAVAN MODEM (VoIP) - TCENTAR':'FAULT MODEM (VoIP) - TCENTAR',
'NEISPRAVAN MODEM (GRMLJAVINA) - TCENTAR':'DEFECTIVE MODEM (THUNDERSTORM) - TCENTAR',
'NE RADI FAX':'FAX WORKS',
'Terastream ? neispravan modem':'Terastream? faulty modem',
'NE RADI POS APARAT':'POS APARTMENT DOES NOT WORK',
'Terastream ?  degradacija':'Terastream? degradation',
'TeraStream - povremeni prekidi':'TeraStream - intermittent interruptions',
'NEISPRAVAN MODEM (VoIP) - DISTRIBUTER':'DEFECTIVE MODEM (VoIP) - DISTRIBUTOR'}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
df_tkts_comb_agg_3 = df_tkts_comb_agg_2.replace(to_replace=mapping,subset=['LABEL'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
df_tkts_comb_agg_3.select(F.col('LABEL')).distinct().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|               LABEL|
+--------------------+
|  2. STB SE NE BOOTA|
|DTH - OCCASIONALL...|
|DEFECTIVE REMOTE ...|
| ADDITIONAL SERVICES|
|DTH - NO EARTH CH...|
| CONSTANTLY OCCUPIED|
|WIRELESS DOES NOT...|
|STB IS OCCASIONAL...|
|DTH - DAMAGED ANT...|
|  DTH - PAIR FAILURE|
|DTH - CHANNEL CHA...|
|DTH - NO SATELLIT...|
|INTERFERENCE AT T...|
|DTH - new transpo...|
|DTH - DEFECTIVE D...|
|       DEFECTIVE STB|
+--------------------+

In [24]:
df_tkts_comb_agg_3.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-------------------+--------------------+
| ASSET_ID|         START_DATE|               LABEL|
+---------+-------------------+--------------------+
|100424784|2021-03-26 16:03:55| CONSTANTLY OCCUPIED|
|101866361|2020-04-04 20:13:56| CONSTANTLY OCCUPIED|
|102594887|2020-09-22 14:33:38| CONSTANTLY OCCUPIED|
|103232787|2020-11-27 08:09:41|DEFECTIVE REMOTE ...|
|103318378|2021-01-18 17:31:26|WIRELESS DOES NOT...|
+---------+-------------------+--------------------+
only showing top 5 rows

In [25]:
path = '/user/tsystems_vkumar/dsl_tickets/df_tkts_non_dsl_sel_final.parquet'
df_tkts_comb_agg_3.repartition(1).write.format('parquet').mode('overwrite').option('header','true').save(path)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…